# (Quick) Python introduction and horizon mask effect

<table>
  <tr>
    <td><img src="https://raw.githubusercontent.com/AlexandreHugoMathieu/pvfault_detection_solar_academy/refs/heads/master/notebooks/logos/CSTB_logo.jpg" width="200" />
    <td><img src="https://raw.githubusercontent.com/AlexandreHugoMathieu/pvfault_detection_solar_academy/refs/heads/master/notebooks/logos/Heliocity_logo.png" width="200" /> 
    <td><img src="https://raw.githubusercontent.com/AlexandreHugoMathieu/pvfault_detection_solar_academy/refs/heads/master/notebooks/logos/USMB_logo_horizontal.png" width="200" />
  </tr>
</table>

<sub>Author: Alexandre MATHIEU, Phd Student, in collaboration Heliocity / CSTB / USMB (LOCIE). </sub>
<sub>Phd thesis: Methodology development to guarantee building photovoltaic systems' performance including failure modelling</sub>

This notebook introduces how to use python to explore data and calculate the effect of shading with the horizon mask from PVGIS.

It is broken down into two parts:

1. **Python introduction**: The student will follow the python introduction and is invited to execute the cells.
2. **Horizon mask effect**: The student is invited to apply the effect of shading on the horizon mask.

The attendees are very much welcome to modify and create/execute some new notebook cells to explore data and try out new methods.

**Make sure to manually execute the cells (ctrl + enter) yourself one by one**

The students are expected to do the **exercices**.

## I. Python introduction

In this introduction, 4 topics will be tackled:

1. Filtering over a pandas Series or DataFrame
2. Use "If" and Loops commands
3. Make plots
4. Create python functions

In [ ]:
import numpy as np # For math operations
import pandas as pd # To create indexed-timeseries
import matplotlib.pyplot as plt # To plot figures

# Load satellite CAMs irradiance data from an online file
urlw="https://raw.githubusercontent.com/AlexandreHugoMathieu/pvfault_detection_solar_academy/refs/heads/master/data/sat_data.csv"
weather_data = pd.read_csv(urlw, index_col=0)
weather_data.index = pd.to_datetime(weather_data.index).tz_convert("Europe/Paris")  # Convert the index to a datetime index and assign it to local time

print(weather_data.dropna().head(20))  # Show the first 5 lines, (all in W/m2)

In [ ]:
# Load satellite position data calculated from NREL algorithm: I. Reda and A. Andreas, Solar position algorithm for solar radiation applications. Solar Energy, vol. 76, no. 5, pp. 577-589, 2004.
# Stored in an online file
urls= "https://raw.githubusercontent.com/AlexandreHugoMathieu/pvfault_detection_solar_academy/refs/heads/master/data/solarpos_data.csv"
solar_position = pd.read_csv(urls, index_col=0)
solar_position.index = pd.to_datetime(solar_position.index).tz_convert("Europe/Paris")  # Convert the index to a datetime index and assign it to local time

print(solar_position.dropna().head(5))  # Show the first 5 lines, (all in ° degree)

### I.1 Pandas Filtering

When it comes to make calculations faster or select a specific chunk of the DataFrame, filtering can be usefull

In [ ]:
# Filter the dataframe when ghi is over a threshold
filter = weather_data["ghi"] > 100

print(weather_data.loc[filter])

# Plot it
weather_data.loc[filter].plot(marker="o")# Add markers

In [ ]:
# Sometimes we don't see much about the data and it can be interesting to filter on some dates
filter_date = weather_data.index > pd.to_datetime("2022-11-01").tz_localize("CET") # Filter after the 1st of November
weather_data.loc[filter_date].plot()

In [ ]:
# The filters can add up
filter_all = filter | filter_date # or
filter_all2 = filter & filter_date # and
weather_data.loc[filter_all2].plot(marker="o")

In [ ]:
# Note that only one column can also be selected according to a filter with the loc command
print(weather_data.loc[filter_all2, "ghi"].head(5))

#### Exercice

Apply the following filters on weather_data with the condition "or":

- ghi > 800 W/m2
- between the 1st of July and the 1st of August

And plot dhi only

## I.2. "If" and loops

"If" allows to verify and execute actions with an assertion.

On the other hand, loops enable to loop over all elements of a list or DataFrame for instance, to performance action on it.

In [ ]:
# "If" allows to execute the next row (with a tabulation to the right) if the assertion is True
a=1
if a<0: # assertion: is "a" under 0 ?
    print("a is lower than 0") # Line non executed since the assertion above is wrong

In [ ]:
if a>0:
    print("a is greater than 0") # Line  executed since the assertion above is right

In [ ]:
# Sometimes operations can not be applied directly with pd.Series and loops allow to go through each element of a list or serie
my_list = ["element 1","element 2"]
for element in my_list:
    print(element)

In [ ]:
# It can be applied to pd.Series
ts = pd.Series([1,2,3])
for element in ts:
    print(element**2)

In [ ]:
# For dataFrame, one can use the "iterrows" method
df = pd.DataFrame()
df["column1"] = [1,2,3]
df["column2"] = [1, 3, 5]
print(df)

In [ ]:
ts_stock = pd.Series(dtype=float) # Prepare the recipient to store the values

for index, row in df.iterrows(): # Loop over all rows of the dataframe one by one
    print(row["column1"] + row["column2"])
    ts_stock.loc[index] = row["column1"] + row["column2"]

In [ ]:
print(ts_stock)

#### Exercice

Create a new pd.Serie timeserie from df which have the same indexes and has a 1 in its row value if the sum over the df row is greater than 4, otherwise, assign 0

### I.3. Plots

In [ ]:
# Here is an example about how to make a plot with the matplotlib.pyplot library
plt.plot(weather_data["ghi"], weather_data["dni"], linewidth=0, marker="o")

plt.xlabel('GHI [W/m2]')
plt.ylabel('DNI [W/m2]')
plt.title("My title")

### I.4. Functions

Functions enable package small algorithms which are easy to apply and to re-use

In [ ]:
# Define the function "" with the "def" command and a small increment tab to the right
def cos_aoi_function(beta, azimuth, sun_elevation, sun_azimuth):
    cos_aoi = np.cos(beta* np.pi/180)*np.sin(sun_elevation*np.pi/180) +np.sin(beta*np.pi/180)*np.cos(sun_elevation*np.pi/180)*np.cos((azimuth-sun_azimuth)*np.pi/180)
    return cos_aoi

# Installation orientation
beta = 20 # tilt [°]
azimuth = 180 # azimuth [°]
sun_elevation = 72 # sun elevation [°]
s_azimuth = 173.9 # sun azimuth [°]

cos_aoi = cos_aoi_function(beta, azimuth, sun_elevation, s_azimuth)
print(cos_aoi)

#### Exercice

Apply the function cos_aoi_function() on the pd.DataFrame "solar_position"


## II Mask Horizon effect

<span style="color: red"> Add the horizon file from PVGIS in the "data" folder by browsing on the left pan and indicate the filename below </span>

In [ ]:
# Indicate the csv-file path
file =  "https://raw.githubusercontent.com/AlexandreHugoMathieu/pvfault_detection_solar_academy/refs/heads/master/data/horizon_45.181_5.714.csv" # for instance: "content/horizon_45.181_5.714.csv"

In [ ]:
# Import the file into a pandas DataFrame  (skip some rows and add some arguments to make the dataframe clean))
data = pd.read_csv(file, sep='\t', skiprows=3, skipfooter=8, engine="python").dropna(axis=1)

# The following lines do some operations
data = data[["A", "H_hor"]] # Keep only the two relevant columns 'A' azimuth and "H_hor": Horizon line
data.columns = ["azimuth", "elevation"] # Rename the columns
data["azimuth"] = data["azimuth"] + 180 # Change the convention (to have the convention with 0° is North, 90° East etc...)
data_ts = data.set_index("azimuth").reindex(np.arange(0,360)).interpolate()["elevation"] #

# Print the 5 fist lines
print(data_ts.head())  # pd.Series

#### Exercice 

a. Plot the elevation as function of the azimuth with matplotlib

In [ ]:
# Develop your code here

b. Apply the shading effect on the direct POA component, ie if the sun elevation is under the horizon line, POAb = 0 W/m2

Advice: You might want to create a loop over the dataframe which contains the POA data and calculate the shading elevation for each of its time step in a new column

In [ ]:
# Develop your code here